In [56]:
import boto3
from tqdm.notebook import tqdm
import pandas as pd
from stringcase import pascalcase

In [28]:
s3 = boto3.client('s3')

In [29]:
pag = s3.get_paginator('list_objects_v2')

In [30]:
page_iter = pag.paginate(Bucket='ft-modelling-lbeta', Prefix='cxr-nih/')

In [31]:
contents = []
for page in tqdm(page_iter):
    if 'Contents' in page:
        contents.extend(page['Contents'])
'Done'

'Done'

In [32]:
len(contents)

113179

In [107]:
df = pd.DataFrame.from_records(contents)

In [108]:
df['Filename'] = df['Key'].apply(lambda key: key.split('/')[-1])

In [109]:
df.set_index('Filename', drop=True, inplace=True)

In [110]:
df = df.drop(columns=['LabelTestRadConsensusVoted3', 'IsTestRadConsensusVoted3'])

KeyError: "['LabelTestRadConsensusVoted3' 'IsTestRadConsensusVoted3'] not found in axis"

In [111]:
df.columns

Index(['Key', 'LastModified', 'ETag', 'Size', 'StorageClass'], dtype='object')

In [112]:
orig_test_df = pd.read_csv('/Users/eoin/workspace/fourTheorem/lbeta/CADLab/CXR-Binary-Classifier/test_list.txt', names=['Filename'], index_col='Filename')
orig_test_df['IsOrigTest'] = True
orig_train_df = pd.read_csv('/Users/eoin/workspace/fourTheorem/lbeta/CADLab/CXR-Binary-Classifier/train_val_list.txt', names=['Filename'], index_col='Filename')
orig_train_df['IsOrigTrain'] = True
tang_set_keys = [
    's3://ft-modelling-lbeta/cxr-nih/tang/test_rad_consensus_voted3.txt',
    's3://ft-modelling-lbeta/cxr-nih/tang/test_attending_rad.txt',
    's3://ft-modelling-lbeta/cxr-nih/tang/train.txt',
]

tang_set_dfs = [
    pd.read_csv(key, names=[
        'Filename', 
        'Label' + pascalcase(key.split('/')[-1].split('.')[0])
    ], sep=' ', index_col='Filename').assign(**{'Is' + pascalcase(key.split('/')[-1].split('.')[0]): True})
    for key in tang_set_keys
]

In [115]:
for tang_df in tang_set_dfs:
    df = df.join(tang_df)

df = df.join(orig_test_df)
df = df.join(orig_train_df)

In [116]:
for col in [col for col in df.columns if col.startswith('Is')]:
    df[col].fillna(value=False, inplace=True)
for col in [col for col in df.columns if col.startswith('Label')]:
    df[col].fillna(value=-1, inplace=True)

In [117]:
df.LabelTestRadConsensusVoted3.unique()

array([-1.,  1.,  0.])

In [121]:
len(df[df.IsTrain])

8668

In [126]:
df.tail()

,Key,LastModified,ETag,Size,StorageClass,LabelTestRadConsensusVoted3,IsTestRadConsensusVoted3,LabelTestAttendingRad,IsTestAttendingRad,LabelTrain,IsTrain,IsOrigTest,IsOrigTrain
Filename,,,,,,,,,,,,,
00030802_000.png,cxr-nih/images_12/images/00030802_000.png,2020-11-14 22:10:23+00:00,"""8cd2e6bcb0340495cb77af2c6bc8cf6b""",390450,STANDARD,-1.0,False,-1.0,False,-1.0,False,True,False
00030803_000.png,cxr-nih/images_12/images/00030803_000.png,2020-11-14 22:10:23+00:00,"""7a42fdade3da297bc789c8df2fbb1bf0""",362506,STANDARD,0.0,True,0.0,True,-1.0,False,True,False
00030804_000.png,cxr-nih/images_12/images/00030804_000.png,2020-11-14 22:10:23+00:00,"""a197ba386e739907d96c69bccb00303b""",381994,STANDARD,-1.0,False,-1.0,False,-1.0,False,True,False
00030805_000.png,cxr-nih/images_12/images/00030805_000.png,2020-11-14 22:10:23+00:00,"""6a8a4dec4d415d1cbf01fe2c09727da8""",668026,STANDARD,-1.0,False,-1.0,False,-1.0,False,True,False
index.csv,cxr-nih/index.csv,2020-11-15 11:05:20+00:00,"""29bbaea34026c53c606f0291066f7d8e-4""",17179417,STANDARD,-1.0,False,-1.0,False,-1.0,False,False,False


In [138]:
df = df.drop('index.csv')

In [140]:
df.to_csv('s3://ft-modelling-lbeta/cxr-nih/index.csv', header=True)

In [139]:
df.to_csv('/Users/eoin/workspace/fourTheorem/lbeta/CADLab/CXR-Binary-Classifier/index.csv', header=True)

In [8]:
import boto3
import pandas as pd
from io import BytesIO, StringIO

In [141]:
s3 = boto3.client('s3')

In [142]:
index_csv_obj = BytesIO()
s3.download_fileobj(Bucket='ft-modelling-lbeta', Key='cxr-nih/index.csv', Fileobj=index_csv_obj)
index_csv_obj.seek(0)

0

In [143]:
index_df = pd.read_csv(index_csv_obj)

In [154]:
len(index_df)

113178

In [144]:
index_df.head()

,Filename,Key,LastModified,ETag,Size,StorageClass,LabelTestRadConsensusVoted3,IsTestRadConsensusVoted3,LabelTestAttendingRad,IsTestAttendingRad,LabelTrain,IsTrain,IsOrigTest,IsOrigTrain
0,00000001_000.png,cxr-nih/images_01/images/00000001_000.png,2020-11-14 21:40:50+00:00,"""9ed48cef82776548c0f4e074f94c11e5""",457171,STANDARD,-1.0,False,-1.0,False,-1.0,False,False,True
1,00000001_001.png,cxr-nih/images_01/images/00000001_001.png,2020-11-14 21:40:50+00:00,"""96f0ea563a3c5814edc1f769cce5ae02""",444655,STANDARD,-1.0,False,-1.0,False,-1.0,False,False,True
2,00000001_002.png,cxr-nih/images_01/images/00000001_002.png,2020-11-14 21:40:50+00:00,"""ce16f474cf68ee9e402ae25515255137""",351772,STANDARD,-1.0,False,-1.0,False,-1.0,False,False,True
3,00000002_000.png,cxr-nih/images_01/images/00000002_000.png,2020-11-14 21:40:50+00:00,"""cd62c75e25b0969232ba67819e314332""",402332,STANDARD,-1.0,False,-1.0,False,-1.0,False,False,True
4,00000003_000.png,cxr-nih/images_01/images/00000003_000.png,2020-11-14 21:40:50+00:00,"""b8fa05b9e43029ce6ac86f928026d739""",450000,STANDARD,-1.0,False,-1.0,False,-1.0,False,True,False


In [146]:
index_df.to

rec.array([(0, '00000001_000.png', 'cxr-nih/images_01/images/00000001_000.png', '2020-11-14 21:40:50+00:00', '"9ed48cef82776548c0f4e074f94c11e5"', 457171, 'STANDARD', -1., False, -1., False, -1., False, False,  True),
           (1, '00000001_001.png', 'cxr-nih/images_01/images/00000001_001.png', '2020-11-14 21:40:50+00:00', '"96f0ea563a3c5814edc1f769cce5ae02"', 444655, 'STANDARD', -1., False, -1., False, -1., False, False,  True),
           (2, '00000001_002.png', 'cxr-nih/images_01/images/00000001_002.png', '2020-11-14 21:40:50+00:00', '"ce16f474cf68ee9e402ae25515255137"', 351772, 'STANDARD', -1., False, -1., False, -1., False, False,  True),
           (3, '00000002_000.png', 'cxr-nih/images_01/images/00000002_000.png', '2020-11-14 21:40:50+00:00', '"cd62c75e25b0969232ba67819e314332"', 402332, 'STANDARD', -1., False, -1., False, -1., False, False,  True),
           (4, '00000003_000.png', 'cxr-nih/images_01/images/00000003_000.png', '2020-11-14 21:40:50+00:00', '"b8fa05b9e43029ce6

In [153]:
for index, row in index_df.drop(columns=['LastModified', 'StorageClass']).iterrows():
    print(row.to_dict())
    break

{'Filename': '00000001_000.png', 'Key': 'cxr-nih/images_01/images/00000001_000.png', 'ETag': '"9ed48cef82776548c0f4e074f94c11e5"', 'Size': 457171, 'LabelTestRadConsensusVoted3': -1.0, 'IsTestRadConsensusVoted3': False, 'LabelTestAttendingRad': -1.0, 'IsTestAttendingRad': False, 'LabelTrain': -1.0, 'IsTrain': False, 'IsOrigTest': False, 'IsOrigTrain': True}
